In [ ]:
#@title 📦 Cell 1: 安装依赖 + 挂载 Google Drive
!pip install spandrel gdown -q

from google.colab import drive
drive.mount('/content/drive')

print("✅ 依赖安装完成，Drive 已挂载")

In [ ]:
#@title ⚙️ Cell 2: 配置参数
import os

# ============ 模型选择 ============
#@markdown ### 模型选择
MODEL_CHOICE = "4x-Nomos8kDAT"  #@param ["4x-Nomos8kDAT", "4x-UltraSharp", "4x-Nomos8kSCSRFormer"]

# 模型下载信息 (Google Drive ID + 预期大小 MB)
MODEL_INFO = {
    "4x-UltraSharp": {
        "gdrive_id": None,  # 用 HuggingFace
        "url": "https://huggingface.co/datasets/Kizi-Art/Upscale/resolve/main/4x-UltraSharp.pth",
        "size_mb": 67,
    },
    "4x-Nomos8kDAT": {
        "gdrive_id": "1JRwXYeuMBIsyeNfsTfeSs7gsHqCZD7xn",
        "url": None,
        "size_mb": 295,
    },
    "4x-Nomos8kSCSRFormer": {
        "gdrive_id": "1VpMity5vqrEN7YFaPwawhVTdWYdJK8DG",
        "url": None,
        "size_mb": 185,
    },
}

MODEL_DIR = "/content/models"
MODEL_NAME = f"{MODEL_CHOICE}.pth"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME)
EXPECTED_SIZE_MB = MODEL_INFO[MODEL_CHOICE]["size_mb"]

# ============ 放大配置 ============
#@markdown ### 放大配置
TARGET_SCALE = 8          #@param {type:"slider", min:4, max:16, step:4}
MAX_SIDE_LENGTH = 8192    #@param {type:"slider", min:4096, max:16384, step:1024}
TILE_SIZE = 512           #@param {type:"slider", min:128, max:1024, step:128}
TILE_OVERLAP = 32         # 分块重叠像素

# ============ 输出格式 ============
#@markdown ### 输出格式
OUTPUT_FORMAT = "jpg"    #@param ["png", "jpg", "both"]

# ============ 路径配置 (直接用 Drive) ============
#@markdown ### 路径配置
DRIVE_BASE = "/content/drive/MyDrive/upscale"  #@param {type:"string"}
INPUT_DIR = f"{DRIVE_BASE}/input"
OUTPUT_DIR = f"{DRIVE_BASE}/output"

# 创建目录
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"🤖 模型: {MODEL_CHOICE} (~{EXPECTED_SIZE_MB} MB)")
print(f"📁 输入目录: {INPUT_DIR}")
print(f"📁 输出目录: {OUTPUT_DIR}")
print(f"🎯 目标放大倍数: {TARGET_SCALE}x")
print(f"🧩 分块大小: {TILE_SIZE}")
print(f"💾 输出格式: {OUTPUT_FORMAT.upper()}")
print("\n💡 提示: 把图片放到 Drive 的 upscale/input 文件夹，然后运行后续 Cell")

In [ ]:
#@title 📂 Cell 3: 检查输入文件夹
import glob

extensions = ['*.png', '*.jpg', '*.jpeg', '*.webp', '*.bmp']
image_files = []
for ext in extensions:
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext)))
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext.upper())))

if not image_files:
    print(f"❌ 输入文件夹为空: {INPUT_DIR}")
    print("💡 请先把图片放到 Google Drive 的 upscale/input 文件夹")
else:
    print(f"✅ 找到 {len(image_files)} 张图片:")

    for f in image_files:
        print(f"  📷 {os.path.basename(f)}")

In [ ]:
#@title 📥 Cell 4: 下载模型
import os
import gdown

def download_model():
    info = MODEL_INFO[MODEL_CHOICE]
    
    if info["gdrive_id"]:
        # 使用 gdown 从 Google Drive 下载
        print(f"⬇️ 正在从 Google Drive 下载 {MODEL_NAME}...")
        gdown.download(id=info["gdrive_id"], output=MODEL_PATH, quiet=False)
    else:
        # 使用 wget 从 URL 下载
        print(f"⬇️ 正在下载 {MODEL_NAME}...")
        url = info["url"]
        !wget -q --show-progress -O "{MODEL_PATH}" "{url}"

# 检查是否需要下载
need_download = False
min_size = EXPECTED_SIZE_MB * 0.9  # 允许 10% 误差

if not os.path.exists(MODEL_PATH):
    need_download = True
else:
    size_mb = os.path.getsize(MODEL_PATH) / (1024 * 1024)
    if size_mb < min_size:
        print(f"⚠️ 模型文件不完整 ({size_mb:.1f} MB < {min_size:.0f} MB)，重新下载...")
        os.remove(MODEL_PATH)
        need_download = True

if need_download:
    download_model()
    print(f"✅ 模型已下载到 {MODEL_PATH}")
else:
    print(f"✅ 模型已存在: {MODEL_PATH}")

# 检查文件大小
size_mb = os.path.getsize(MODEL_PATH) / (1024 * 1024)
print(f"📦 模型大小: {size_mb:.1f} MB (预期: ~{EXPECTED_SIZE_MB} MB)")

if size_mb < min_size:
    print(f"❌ 模型文件可能损坏，请删除后重新下载")
    print(f"   运行: !rm \"{MODEL_PATH}\"")

In [ ]:
#@title 🔧 Cell 5: 加载模型
import torch
import spandrel

print("🔄 正在加载模型...")

# 检查 CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 使用设备: {device}")

# 加载模型
try:
    model_descriptor = spandrel.ModelLoader(device=device).load_from_file(MODEL_PATH)
except Exception as e:
    print(f"\n❌ 模型加载失败: {e}")
    print(f"⚠️ 模型文件可能损坏或下载不完整")
    
    # 删除损坏的模型文件
    if os.path.exists(MODEL_PATH):
        os.remove(MODEL_PATH)
        print(f"🗑️ 已删除损坏的模型文件: {MODEL_PATH}")
    
    print("\n💡 请重新运行 Cell 4 下载模型")
    print("🔌 正在断开运行时以节省点数...")
    
    from google.colab import runtime
    runtime.unassign()
    raise  # 确保停止执行

model = model_descriptor.model

# 只有 ESRGAN 架构支持 FP16，DAT/SRFormer 需要 FP32
USE_FP16 = "UltraSharp" in MODEL_CHOICE  # ESRGAN 架构
if USE_FP16:
    model.half()
    print("⚡ 使用 FP16 (半精度)")
else:
    print("🔢 使用 FP32 (全精度) - DAT/SRFormer 架构需要")

model.eval()

MODEL_SCALE = model_descriptor.scale  # 模型原生放大倍数 (4x)

print(f"✅ 模型加载完成")
print(f"📐 模型原生倍数: {MODEL_SCALE}x")
print(f"🎯 目标倍数: {TARGET_SCALE}x → 需要迭代 {TARGET_SCALE // MODEL_SCALE} 次")

In [ ]:
#@title 🛠️ Cell 6: 定义处理函数
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import math

def pil_to_torch(img: Image.Image) -> torch.Tensor:
    """PIL Image (RGB) → Torch Tensor (NCHW, BGR, 0~1)"""
    img_np = np.array(img).astype(np.float32) / 255.0
    # RGB → BGR
    if img_np.ndim == 3 and img_np.shape[2] == 3:
        img_np = img_np[:, :, ::-1].copy()  # copy() 解决负 stride 问题
    # HWC → CHW → NCHW
    img_tensor = torch.from_numpy(img_np.transpose(2, 0, 1).copy()).unsqueeze(0)
    img_tensor = img_tensor.to(device)
    if USE_FP16:
        img_tensor = img_tensor.half()
    return img_tensor

def torch_to_pil(tensor: torch.Tensor) -> Image.Image:
    """Torch Tensor (NCHW, BGR) → PIL Image (RGB)"""
    img_np = tensor.squeeze(0).float().cpu().clamp(0, 1).numpy()
    # CHW → HWC
    img_np = img_np.transpose(1, 2, 0)
    # BGR → RGB
    img_np = img_np[:, :, ::-1].copy()  # copy() 解决负 stride 问题
    img_np = (img_np * 255).astype(np.uint8)
    return Image.fromarray(img_np)

@torch.inference_mode()
def tiled_upscale(img: Image.Image, tile_size: int = 512, overlap: int = 32) -> Image.Image:
    """
    分块放大，避免显存爆炸
    参考: stable-diffusion-webui/modules/upscaler_utils.py
    """
    width, height = img.size
    
    # 计算分块数量
    tiles_x = math.ceil(width / (tile_size - overlap))
    tiles_y = math.ceil(height / (tile_size - overlap))
    
    # 输出尺寸
    out_width = width * MODEL_SCALE
    out_height = height * MODEL_SCALE
    output = Image.new('RGB', (out_width, out_height))
    
    # 放大后的 overlap
    scaled_overlap = overlap * MODEL_SCALE
    
    for y in range(tiles_y):
        for x in range(tiles_x):
            # 计算输入 tile 的坐标
            x1 = x * (tile_size - overlap)
            y1 = y * (tile_size - overlap)
            x2 = min(x1 + tile_size, width)
            y2 = min(y1 + tile_size, height)
            
            # 确保 tile 尺寸一致（边缘情况）
            if x2 - x1 < tile_size and x1 > 0:
                x1 = max(0, x2 - tile_size)
            if y2 - y1 < tile_size and y1 > 0:
                y1 = max(0, y2 - tile_size)
            
            # 裁切 tile
            tile = img.crop((x1, y1, x2, y2))
            
            # 放大
            tile_tensor = pil_to_torch(tile)
            upscaled_tensor = model(tile_tensor)
            upscaled_tile = torch_to_pil(upscaled_tensor)
            
            # 计算输出坐标
            out_x1 = x1 * MODEL_SCALE
            out_y1 = y1 * MODEL_SCALE
            out_x2 = x2 * MODEL_SCALE
            out_y2 = y2 * MODEL_SCALE
            
            # 计算有效区域（去除重叠部分，除了边缘）
            paste_x1 = 0 if x == 0 else scaled_overlap // 2
            paste_y1 = 0 if y == 0 else scaled_overlap // 2
            paste_x2 = upscaled_tile.width if x == tiles_x - 1 else upscaled_tile.width - scaled_overlap // 2
            paste_y2 = upscaled_tile.height if y == tiles_y - 1 else upscaled_tile.height - scaled_overlap // 2
            
            # 裁切有效区域
            valid_tile = upscaled_tile.crop((paste_x1, paste_y1, paste_x2, paste_y2))
            
            # 粘贴到输出
            final_x = out_x1 + paste_x1
            final_y = out_y1 + paste_y1
            output.paste(valid_tile, (final_x, final_y))
    
    return output

def multi_scale_upscale(img: Image.Image, target_scale: int) -> Image.Image:
    """
    放大图片到目标倍数，不超过 MAX_SIDE_LENGTH
    参考: stable-diffusion-webui Extras 逻辑
    """
    # 中间图最大像素数限制 (防止计算时间爆炸)
    # 1亿像素 ≈ 10000x10000，DAT 处理约需 2-3 分钟
    MAX_INTERMEDIATE_PIXELS = 100_000_000  # 1亿像素
    
    orig_w, orig_h = img.size
    max_orig_side = max(orig_w, orig_h)
    
    # 计算实际可用的放大倍数（不超过 MAX_SIDE_LENGTH）
    max_possible_scale = MAX_SIDE_LENGTH / max_orig_side
    actual_scale = min(target_scale, max_possible_scale)
    
    # 计算目标尺寸 (对齐到8像素，参考 SD WebUI)
    target_w = int((orig_w * actual_scale) // 8 * 8)
    target_h = int((orig_h * actual_scale) // 8 * 8)
    
    print(f"  🎯 目标尺寸: {target_w}x{target_h} ({actual_scale:.2f}x)")
    
    # 如果原图已经够大
    if actual_scale <= 1:
        print(f"  ⚠️ 原图已接近最大边长，无需放大")
        return img
    
    # 迭代放大，但限制中间像素数
    current = img
    current_scale = 1.0
    iteration = 0
    
    while current_scale < actual_scale:
        # 检查下一次放大后的尺寸和像素数
        next_w = current.width * MODEL_SCALE
        next_h = current.height * MODEL_SCALE
        next_pixels = next_w * next_h
        
        # 如果下一次放大会超过像素数限制，提前停止
        if next_pixels > MAX_INTERMEDIATE_PIXELS:
            print(f"  ⚠️ 下次放大 ({next_w}x{next_h} = {next_pixels/1e6:.0f}M 像素) 超过限制，跳过")
            break
        
        iteration += 1
        print(f"  📐 迭代 {iteration}: {current.width}x{current.height} → {next_w}x{next_h}")
        current = tiled_upscale(current, TILE_SIZE, TILE_OVERLAP)
        current_scale *= MODEL_SCALE
    
    # 最后 resize 到精确的目标尺寸 (只缩小，不放大)
    if current.size != (target_w, target_h):
        if current.width >= target_w and current.height >= target_h:
            # ESRGAN 结果比目标大，缩小到目标尺寸
            print(f"  🔄 Resize (缩小): {current.width}x{current.height} → {target_w}x{target_h}")
            current = current.resize((target_w, target_h), Image.LANCZOS)
        else:
            # ESRGAN 结果比目标小，不用传统插值放大，直接输出
            print(f"  ⚠️ 跳过传统放大，输出 ESRGAN 结果: {current.width}x{current.height}")
    
    return current

print("✅ 处理函数已定义")

In [ ]:
#@title 🚀 Cell 7: 批量处理 → 保存到 Drive
import glob
import time

# 获取所有图片
extensions = ['*.png', '*.jpg', '*.jpeg', '*.webp', '*.bmp']
image_files = []
for ext in extensions:
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext)))
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext.upper())))

if not image_files:
    print("❌ 没有找到图片，请先运行 Cell 3 上传图片")
else:
    print(f"📷 找到 {len(image_files)} 张图片")
    print(f"🎯 目标倍数: {TARGET_SCALE}x")
    print(f"📁 输出目录: {OUTPUT_DIR}")
    print("=" * 50)
    
    start_time = time.time()
    failed = False
    
    for idx, img_path in enumerate(image_files, 1):
        filename = os.path.basename(img_path)
        name, ext = os.path.splitext(filename)
        
        print(f"\n[{idx}/{len(image_files)}] 处理: {filename}")
        
        try:
            # 加载图片
            img = Image.open(img_path).convert('RGB')
            orig_size = img.size
            print(f"  📐 原始尺寸: {orig_size[0]}x{orig_size[1]}")
            
            # 放大
            upscaled = multi_scale_upscale(img, TARGET_SCALE)
            final_size = upscaled.size
            print(f"  ✅ 最终尺寸: {final_size[0]}x{final_size[1]}")
            
            # 根据配置保存
            if OUTPUT_FORMAT in ("png", "both"):
                png_path = os.path.join(OUTPUT_DIR, f"{name}.png")
                upscaled.save(png_path)
                print(f"  💾 已保存: {name}.png")
            
            if OUTPUT_FORMAT in ("jpg", "both"):
                # JPG: quality=92 + 4:4:4 ≈ Photoshop 质量 11/12
                jpg_path = os.path.join(OUTPUT_DIR, f"{name}.jpg")
                upscaled.save(jpg_path, quality=92, subsampling=0)
                print(f"  💾 已保存: {name}.jpg")
            
        except Exception as e:
            print(f"  ❌ 错误: {e}")
            failed = True
            break  # 遇到错误立即停止
    
    elapsed = time.time() - start_time
    print("\n" + "=" * 50)
    
    if failed:
        print(f"❌ 处理失败，耗时: {elapsed:.1f} 秒")
        print("🔌 正在断开运行时以节省点数...")
        from google.colab import runtime
        runtime.unassign()
    else:
        print(f"🎉 全部完成! 耗时: {elapsed:.1f} 秒")
        print(f"📁 输出位置: {OUTPUT_DIR}")

In [ ]:
#@title 🔌 Cell 8: 断开运行时
from google.colab import runtime

print("⚠️ 即将断开运行时...")
print("📁 输出已保存到 Google Drive，无需等待下载")

runtime.unassign()